In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_artists_origins = pd.read_csv('Datasets/other_datasets/df_artists_origins.csv')
df_ratings = pd.read_csv('Datasets/df_ratings.csv')

In [3]:
df_ratings

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
0,1996,Less Than Jake,Losing Streak,3.90,414,37.08,17,2.18,"['Ska', 'Punk']"
1,2002,Sparta,Wiretap Scars,3.79,431,47.62,12,3.97,"['Indie Rock', 'Alternative Rock']"
2,2003,Muse,Absolution,3.99,4411,52.18,14,3.73,['Alternative Rock']
3,1999,Muse,Showbiz,3.50,2181,49.65,12,4.14,"['Alternative Rock', 'Prog Rock']"
4,2002,Finch,What It Is to Burn,3.69,864,61.18,13,4.71,"['Emo', 'Post-Hardcore', 'Alternative Rock']"
...,...,...,...,...,...,...,...,...,...
11143,1988,Wire,A Bell Is A Cup Until It Is Struck,3.27,59,43.65,10,4.36,['New Wave']
11144,1990,Wire,Manscape,2.60,20,63.43,13,4.88,['Post-Punk']
11145,1991,Wire,The Drill,2.46,20,55.39,9,6.15,['New Wave']
11146,2003,Zao,All Else Failed,3.07,117,41.60,8,5.20,"['Hardcore', 'Death Metal']"


In [23]:
artist = 'Big Big Train'
artist.lower().replace(' ', '')

'bigbigtrain'

In [21]:
artists = df_ratings['artist'].unique()
artists[:10]

array(['Less Than Jake', 'Sparta', 'Muse', 'Finch',
       'Queens of the Stone Age', 'Transplants', 'Dream Theater',
       'A Perfect Circle', 'Rooney', 'AFI'], dtype=object)

In [169]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv')
artists = df_ratings['artist'].unique()

# create empty lists
artists_list = []
titles_list = []
title_changed_list = []
album_length_list = []
prices_list = []
years_list = []

for artist in artists[:1]:
    artist_clean = artist.lower().replace(' ', '')

    try:
        url = f'https://{artist_clean}.bandcamp.com/music'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        location = soup.select('div p span', class_="location secondaryText")[1].text
        print(f"{artist_clean} - {location}")

        releases = soup.select('#music-grid li a p')
        for i in releases:
            title = i.text.replace('\n', '').strip()
            print(title)
            title_changed = (
                title.replace('(', '')
                     .replace(')', '')
                     .replace('[', '')
                     .replace(']', '')
                     .replace('/', ' ')
                     .replace("'", '')
                     .replace('& ', '')
                     .replace('feat.', 'feat')
                     .replace(' ', '-')
                    .lower()
            )
            artists_list.append(artist)
            titles_list.append(title)
            title_changed_list.append(title_changed)

            try:
                url = f'https://{artist_clean}.bandcamp.com/album/{title_changed}'
                response = requests.get(url)
                soup = BeautifulSoup(response.content, "html.parser")
                price = soup.select('#trackInfoInner > ul > li.buyItem.digital > div.ft > h4.ft.compound-button.main-button > span > span.base-text-color')[0].text.replace('$', '')
                prices_list.append(float(price))
                release_date = soup.select('#trackInfoInner > div.tralbumData.tralbum-credits')
                year = release_date[0].text.strip().split(', ')[1]
                years_list.append(year)
                songs_table = soup.select('table', class_='track_list track_table')[1]('span')
                song_durations = []

                for i in range(1, len(songs_table), 2):
                    song_duration = songs_table[i].text.strip()
                    try:
                        minutes, seconds = map(int, song_duration.split(':'))
                        song_duration_minutes = minutes + seconds/60
                        song_durations.append(song_duration_minutes)
                    except:
                        pass
                    album_length = round(sum(song_durations), 2)
                album_length_list.append(album_length)
            except: 
                print("Couldn't find the album")
                album_length_list.append(np.nan)
                prices_list.append(np.nan)
                years_list.append(np.nan)
            print(album_length)
    except:
        artists_list.append(artist)
        titles_list.append(np.nan)
        title_changed_list.append(np.nan)
        print(f"{artist_clean} - Maybe this artist doesn't have a bandcamp page")
    
# Create a DataFrame with the results
df_bandcamp = pd.DataFrame({'artist': artists_list
                                , 'title': titles_list
                                , 'title_changed': title_changed_list
                                , 'year': years_list
                                , 'album_length': album_length_list
                                , 'price': prices_list})

lessthanjake - Gainesville, Florida
Uncharted
22.2
Silver Linings (Deluxe)
54.5
Fat Mike's On Drugs (again)
2.37
Silver Linings
36.68
Lost At Home Sessions
Couldn't find the album
36.68
Sound The Alarm EP
20.82
American Idle
5.82
Do The Math
5.75
See The Light
27.83
Greetings & Salutations
34.28
Borders & Boundaries (Reissue)
47.87


In [170]:
df_bandcamp['price_per_minute'] = round(df_bandcamp['price'] / df_bandcamp['album_length'], 2)
df_bandcamp

,artist,title,title_changed,year,album_length,price,price_per_minute
0,Less Than Jake,Uncharted,uncharted,2024,22.20,7.0,0.32
1,Less Than Jake,Silver Linings (Deluxe),silver-linings-deluxe,2022,54.50,9.0,0.17
2,Less Than Jake,Fat Mike's On Drugs (again),fat-mikes-on-drugs-again,2022,2.37,1.0,0.42
3,Less Than Jake,Silver Linings,silver-linings,2020,36.68,7.0,0.19
4,Less Than Jake,Lost At Home Sessions,lost-at-home-sessions,NaN,NaN,NaN,NaN
5,Less Than Jake,Sound The Alarm EP,sound-the-alarm-ep,2017,20.82,5.0,0.24
6,Less Than Jake,American Idle,american-idle,2015,5.82,2.0,0.34
7,Less Than Jake,Do The Math,do-the-math,2014,5.75,2.0,0.35
8,Less Than Jake,See The Light,see-the-light,2013,27.83,10.0,0.36
9,Less Than Jake,Greetings & Salutations,greetings-salutations,2013,34.28,10.0,0.29


In [146]:
df_bandcamp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         11 non-null     object 
 1   title          11 non-null     object 
 2   title_changed  11 non-null     object 
 3   album_length   10 non-null     float64
 4   price          10 non-null     object 
dtypes: float64(1), object(4)
memory usage: 572.0+ bytes


In [168]:
url = f'https://sparta.bandcamp.com/album/sparta'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
release_date = soup.select('#trackInfoInner > div.tralbumData.tralbum-credits')
year = release_date[0].text.strip().split(', ')[1]
year

'2022'

In [123]:
df_ratings[df_ratings['artist']=='Sparta']

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
1,2002,Sparta,Wiretap Scars,3.79,431,47.62,12,3.97,"['Indie Rock', 'Alternative Rock']"
933,2004,Sparta,Porcelain,3.59,343,57.58,14,4.11,['Indie Rock']
3911,2006,Sparta,Threes,3.08,253,49.55,12,4.13,"['Alternative Rock', 'Indie Rock', 'Post-Hardc..."


In [44]:
df_ratings[df_ratings['artist']=='Less Than Jake']

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
0,1996,Less Than Jake,Losing Streak,3.90,414,37.08,17,2.18,"['Ska', 'Punk']"
72,2003,Less Than Jake,Anthem,3.70,472,43.68,18,2.43,"['Ska', 'Punk']"
154,2000,Less Than Jake,Borders and Boundaries,3.55,238,40.97,15,2.73,['Ska']
428,2004,Less Than Jake,B Is for B-sides,3.49,96,27.77,12,2.31,"['Ska', 'Punk']"
928,1998,Less Than Jake,Hello Rockview,3.97,593,37.83,14,2.70,"['Punk', 'Ska']"
1589,1995,Less Than Jake,Pezcore,3.73,191,50.07,21,2.38,"['Punk', 'Ska']"
2873,2006,Less Than Jake,In with the Out Crowd,2.67,279,39.02,12,3.25,"['Pop Rock', 'Punk', 'Ska']"
6485,2008,Less Than Jake,GNV FLA,3.62,238,37.85,15,2.52,"['Ska', 'Punk']"


In [ ]:
df_artists_origins.head(1)

,artist,origin
0,Sparta,"El Paso, Texas"


In [42]:
df_artists_origins[df_artists_origins['artist']=='Less Than Jake']

,artist,origin
27,Less Than Jake,"Gainesville, Florida, U.S."


In [111]:
try:
    url = f'https://lessthanjake.bandcamp.com/music'
    response = requests.get(f"{url}")
    soup = BeautifulSoup(response.content, "html.parser")
    releases = soup.select('#music-grid li a p')
except:
    print("Maybe this artist doesn't have a bandcamp page")

In [116]:
releases[0].text.replace('\n', '').strip()

'Uncharted'

In [ ]:
artist = 'Less Than Jake'
title = 'Uncharted'
query = artist + ' ' + title

    # Define API headers
headers = {"User-Agent": 'Arnau', 
    "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

    # Set the API URL and params
url = "https://api.discogs.com/database/search"
params = {
    "per_page": 100,  # Number of results per page (max 100)
    'type': 'release',
    # 'format': 'album',
    'genre': 'Rock',
    'query': query
}

    # Request the data from Discogs
response = requests.get(url, headers=headers, params=params)
data = response.json()['results']

for album in range(len(data)):
    master_id = data[album]['master_id']

    url_master = f"https://api.discogs.com/masters/{master_id}"
    response = requests.get(url_master, headers=headers)
    time.sleep(1)
    data = response.json()

# a more complex information is the total duration of the album, I have to sum the durations of each song
    number_of_songs = len(data['tracklist'])
    song_durations = []

    for song in range(number_of_songs):
        song_duration = data['tracklist'][song]['duration']
        try:
            minutes = int(song_duration.split(':')[0])
            seconds = int(song_duration.split(':')[1])
            song_duration_minutes = minutes + seconds/60
            song_durations.append(song_duration_minutes)
        except:
            pass
        album_length = round(sum(i for i in song_durations), 2)
        print(album_length)


3657783

In [145]:
data[0]

{'country': 'USA, Canada & Europe',
 'year': '2024',
 'format': ['Vinyl', '12"', 'Single Sided', 'EP', 'Limited Edition'],
 'label': ['Pure Noise Records',
  'GZ Media',
  'Pirates Press',
  'Pure Noise Records'],
 'type': 'release',
 'genre': ['Rock'],
 'style': ['Ska', 'Punk'],
 'id': 32338776,
 'barcode': ['810540037233', '287856E1/A PIRATES PRESS - PNE417'],
 'user_data': {'in_wantlist': False, 'in_collection': False},
 'master_id': 3657783,
 'master_url': 'https://api.discogs.com/masters/3657783',
 'uri': '/release/32338776-Less-Than-Jake-Uncharted',
 'catno': 'PNE417',
 'title': 'Less Than Jake - Uncharted',
 'thumb': 'https://i.discogs.com/zpdqp2JSTlp1xy4LkiaGuTFPQsQrAXtDDxYCyFe3ELE/rs:fit/g:sm/q:40/h:150/w:150/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTMyMzM4/Nzc2LTE3MzIwNjEz/MDctOTg3My5qcGVn.jpeg',
 'cover_image': 'https://i.discogs.com/0Q5896S-CFdFLEGYB0Qn8QEWMXUV4x6SOcx24VHqc18/rs:fit/g:sm/q:90/h:600/w:600/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTMyMzM4/Nzc2LTE3MzIwNjEz/

## **Saving the data**

In [97]:
lists = [('album_id', album_id_list)
        , ('artists', artists_list)
         , ('albums', albums_list)
         , ('avg_rating', avg_rating_list)
         , ('number_of_votes', number_of_votes_list)
         ]

for name, lst in lists:
    print(f'{name}: {len(lst)}') # check that all lists have the same length (even though I had make sure with the previous code)

album_id: 26
artists: 26
albums: 26
avg_rating: 26
number_of_votes: 26


In [98]:
# the new df with the artists I just scrapped
data = pd.DataFrame({'album_id': album_id_list
                    , 'artist': artists_list
                    , 'album': albums_list
                    , 'rating': avg_rating_list
                    , 'votes': number_of_votes_list})
print(f'{data.shape[0]} releases\n{data.artist.nunique()} different artists')

26 releases
22 different artists


In [99]:
print(f'I could webscrap {round(scrapped / count*100, 2)}% of the albums\n{scrapped} of {count} total albums')

I could webscrap 2.6% of the albums
26 of 999 total albums


In [100]:
print(f'{data[data['votes']<10].shape[0]} albums with < 10 votes, meaning {round((data[data['votes']<10].shape[0] / data.shape[0])*100,2)}% of the albums')
print(f'{data[data['votes']>10].shape[0]} of {data.shape[0]} actual valid albums')

26 albums with < 10 votes, meaning 100.0% of the albums
0 of 26 actual valid albums


## **Exporting to csv (and importing in case needed)**

In [107]:
df_ratings.to_csv('Datasets/df_ratings.csv', index=False)

In [2]:
# in case I have to import it again
df_ratings = pd.read_csv('Datasets/df_ratings.csv', keep_default_na=False)